In [36]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [37]:
# mount driv
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# load data

df = pd.read_csv('/content/drive/MyDrive/Datasets/Capstone/Crash_Reporting_-_Drivers_Data.csv')

<ipython-input-38-a35300d00161>:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Datasets/Capstone/Crash_Reporting_-_Drivers_Data.csv')


In [68]:
df.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Name,Off-Road Description,Municipality,...,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,DM8479000T,210020119,Takoma Park Police Depart,Property Damage Crash,05/27/2021 07:40:00 PM,NaN,NaN,NaN,IN PARKING LOT,NaN,...,NaN,0,No,Yes,2017,HINO,TWK,38.987657,-76.987545,"(38.98765667, -76.987545)"
1,MCP2970000R,15045937,MONTGOMERY,Property Damage Crash,09/11/2015 01:29:00 PM,NaN,NaN,NaN,Parking Lot: \n2525 Ennalls Ave,NaN,...,South,5,No,No,2012,TOYOTA,SU,39.039917,-77.053649,"(39.03991652, -77.05364898)"
2,MCP20160036,180040948,Montgomery County Police,Property Damage Crash,08/17/2018 02:25:00 PM,NaN,NaN,NaN,PARKING LOT OF 16246 FREDERICK RD,NaN,...,West,15,No,No,2015,MAZD,TK,38.743373,-77.546997,"(38.743373, -77.54699707)"
3,EJ7879003C,230048975,Gaithersburg Police Depar,Injury Crash,08/11/2023 06:00:00 PM,NaN,NaN,NaN,1 N SUMMIT DRIVE,NaN,...,Unknown,15,No,No,2018,RAM,TK,39.145873,-77.191940,"(39.14587303, -77.19194047)"
4,MCP2967004Y,230070277,Montgomery County Police,Property Damage Crash,12/06/2023 06:42:00 PM,Maryland (State),CONNECTICUT AVE,BALTIMORE ST,NaN,KENSINGTON,...,South,35,No,No,2017,AUDI,A3,39.025170,-77.076333,"(39.02517017, -77.07633333)"


In [40]:
df.columns

Index(['Report Number', 'Local Case Number', 'Agency Name', 'ACRS Report Type',
       'Crash Date/Time', 'Route Type', 'Road Name', 'Cross-Street Name',
       'Off-Road Description', 'Municipality', 'Related Non-Motorist',
       'Collision Type', 'Weather', 'Surface Condition', 'Light',
       'Traffic Control', 'Driver Substance Abuse',
       'Non-Motorist Substance Abuse', 'Person ID', 'Driver At Fault',
       'Injury Severity', 'Circumstance', 'Driver Distracted By',
       'Drivers License State', 'Vehicle ID', 'Vehicle Damage Extent',
       'Vehicle First Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Going Dir', 'Speed Limit',
       'Driverless Vehicle', 'Parked Vehicle', 'Vehicle Year', 'Vehicle Make',
       'Vehicle Model', 'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [41]:
id_cols = ['Report Number', 'Local Case Number', 'Person ID', 'Vehicle ID', 'Agency Name', 'Road Name', 'Location']
df_wo_ids = df.drop(columns=id_cols)

In [42]:
df_wo_ids.shape

(183856, 32)

In [43]:
df_wo_ids.isna().sum()

,0
ACRS Report Type,0
Crash Date/Time,0
Route Type,18036
Cross-Street Name,23186
Off-Road Description,166858
Municipality,164730
Related Non-Motorist,177991
Collision Type,585
Weather,13356
Surface Condition,21230


In [44]:
q1df = df_wo_ids[["Weather", "Light", "Surface Condition", "Injury Severity"]]
q1df.head()

,Weather,Light,Surface Condition,Injury Severity
0,CLEAR,DAYLIGHT,NaN,NO APPARENT INJURY
1,CLEAR,DAYLIGHT,NaN,NO APPARENT INJURY
2,CLEAR,DAYLIGHT,NaN,NO APPARENT INJURY
3,CLEAR,DAYLIGHT,NaN,NO APPARENT INJURY
4,CLEAR,DARK LIGHTS ON,DRY,NO APPARENT INJURY


In [45]:
q1df.isna().sum()

,0
Weather,13356
Light,1445
Surface Condition,21230
Injury Severity,711


## Data Cleaning

In [46]:
def clean_injury_severity(df):
  df['Injury Severity'] = df['Injury Severity'].str.upper()

  # Define mapping to standardize entries
  mapping = {
      'NO APPARENT INJURY': 'NO APPARENT INJURY',
      'POSSIBLE INJURY': 'POSSIBLE INJURY',
      'SUSPECTED MINOR INJURY': 'SUSPECTED MINOR INJURY',
      'SUSPECTED SERIOUS INJURY': 'SUSPECTED SERIOUS INJURY',
      'FATAL INJURY': 'FATAL INJURY'
  }

  # Apply mapping
  df['Injury Severity'] = df['Injury Severity'].replace(mapping)

  severity_mapping = {
      'NO APPARENT INJURY': 'No',
      'POSSIBLE INJURY': 'Yes',
      'SUSPECTED MINOR INJURY': 'Yes',
      'SUSPECTED SERIOUS INJURY': 'Yes',
      'FATAL INJURY': 'Yes'
  }

  # Apply mapping
  df['Injury Severity'] = df['Injury Severity'].map(severity_mapping)
  return df

In [47]:
q1df_new = clean_injury_severity(q1df)

<ipython-input-46-fff5988802ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Injury Severity'] = df['Injury Severity'].str.upper()
<ipython-input-46-fff5988802ca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Injury Severity'] = df['Injury Severity'].replace(mapping)
<ipython-input-46-fff5988802ca>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [48]:
q1df_new.head()

,Weather,Light,Surface Condition,Injury Severity
0,CLEAR,DAYLIGHT,NaN,No
1,CLEAR,DAYLIGHT,NaN,No
2,CLEAR,DAYLIGHT,NaN,No
3,CLEAR,DAYLIGHT,NaN,No
4,CLEAR,DARK LIGHTS ON,DRY,No


In [49]:
q1df_new['Injury Severity']=q1df_new['Injury Severity'].replace({"No": 0, "Yes": 1})

<ipython-input-49-c823cac35d9a>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  q1df_new['Injury Severity']=q1df_new['Injury Severity'].replace({"No": 0, "Yes": 1})
<ipython-input-49-c823cac35d9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1df_new['Injury Severity']=q1df_new['Injury Severity'].replace({"No": 0, "Yes": 1})


In [50]:
weather_mapping = {
    "CLEAR": "CLEAR",
    "Clear": "CLEAR",
    "RAINING": "RAIN",
    "Rain": "RAIN",
    "CLOUDY": "CLEAR",
    "Cloudy": "CLEAR",
    "SNOW": "SNOW",
    "Snow": "SNOW",
    "FOGGY": "CLEAR",
    "Fog, Smog, Smoke": "CLEAR",
    "BLOWING SNOW": "SNOW",
    "Blowing Snow": "SNOW",
    "WINTRY MIX": "SNOW",
    "SLEET": "SNOW",
    "Sleet Or Hail": "SNOW",
    "SEVERE WINDS": "CLEAR",
    "Severe Crosswinds": "CLEAR",
    "BLOWING SAND, SOIL, DIRT": "UNKNOWN",
    "Freezing Rain Or Freezing Drizzle": "RAIN",
    "UNKNOWN": "UNKNOWN",
    "Unknown": "UNKNOWN",
    "OTHER": "UNKNOWN"
}


q1df_new['Weather'] = q1df_new['Weather'].replace(weather_mapping)

<ipython-input-50-71682a028024>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1df_new['Weather'] = q1df_new['Weather'].replace(weather_mapping)


In [51]:
light_condition_mapping = {
    "DAYLIGHT": "LIGHT",
    "Daylight": "LIGHT",
    "DUSK": "LIGHT",
    "Dusk": "LIGHT",
    "DAWN": "LIGHT",
    "Dawn": "LIGHT",
    "DARK LIGHTS ON": "Artificial Light",
    "Dark - Lighted": "Artificial Light",
    "DARK -- UNKNOWN LIGHTING": "DARK",
    "Dark - Unknown Lighting": "DARK",
    "DARK NO LIGHTS": "DARK",
    "Dark - Not Lighted": "DARK",
    "UNKNOWN": "Unknown",
    "Unknown": "Unknown",
    "OTHER": "Unknown",
    "Other": "Unknown"
}


q1df_new['Light'].replace(light_condition_mapping, inplace = True)

<ipython-input-51-3b632764cfc1>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  q1df_new['Light'].replace(light_condition_mapping, inplace = True)
<ipython-input-51-3b632764cfc1>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1df_new['Light'].replace(light_condition_mapping, inplace = True)


In [52]:
q1df_new['Surface Condition'].value_counts()

,count
Surface Condition,
DRY,120577
WET,28431
Dry,8765
Wet,1647
ICE,1058
SNOW,938
UNKNOWN,509
SLUSH,204
OTHER,164


In [53]:
surface_condition_mapping = {
    "DRY": "Dry",
    "Dry": "Dry",
    "WET": "Wet",
    "Wet": "Wet",
    "ICE": "Icy",
    "Ice/Frost": "Icy",
    "Snow": "Snowy",
    "SNOW": "Snowy",
    "SLUSH": "Snowy",
    "Slush": "Snowy",
    "UNKNOWN": "Unknown",
    "Other": "Unknown",
    "OTHER": "Unknown",
    "MUD, DIRT, GRAVEL": "Wet",
    "Mud, Dirt, Gravel": "Wet",
    "SAND": "Wet",
    "Sand": "Wet",
    "WATER(STANDING/MOVING)": "Wet",
    "Water (standing, moving)": "Wet",
    "OIL": "Wet"
}


q1df_new['Surface Condition'].replace(surface_condition_mapping, inplace = True)

<ipython-input-53-4008bc18c6f4>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  q1df_new['Surface Condition'].replace(surface_condition_mapping, inplace = True)
<ipython-input-53-4008bc18c6f4>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1df_new['Surface Condition'].replace(surface_condition_mapping, inplace = True)


In [56]:
q1df_new.head()

,Injury Severity,Weather_RAIN,Weather_SNOW,Weather_UNKNOWN,Light_DARK,Light_LIGHT,Light_Unknown,Surface Condition_Icy,Surface Condition_Snowy,Surface Condition_Unknown,Surface Condition_Wet
0,0.0,0,0,0,0,1,0,0,0,0,0
1,0.0,0,0,0,0,1,0,0,0,0,0
2,0.0,0,0,0,0,1,0,0,0,0,0
3,0.0,0,0,0,0,1,0,0,0,0,0
4,0.0,0,0,0,0,0,0,0,0,0,0


In [55]:
# one hot encoding

q1df_new = pd.get_dummies(q1df_new, columns=['Weather', 'Light', 'Surface Condition'], drop_first=True, dtype = int)

In [57]:
q1df_new.isna().sum()

,0
Injury Severity,711
Weather_RAIN,0
Weather_SNOW,0
Weather_UNKNOWN,0
Light_DARK,0
Light_LIGHT,0
Light_Unknown,0
Surface Condition_Icy,0
Surface Condition_Snowy,0
Surface Condition_Unknown,0


In [58]:
q1df_new.shape

(183856, 11)

In [59]:
q1df_new.dropna(inplace=True)

In [60]:
q1df_new.shape

(183145, 11)

In [61]:
q1df_new.isna().sum()

,0
Injury Severity,0
Weather_RAIN,0
Weather_SNOW,0
Weather_UNKNOWN,0
Light_DARK,0
Light_LIGHT,0
Light_Unknown,0
Surface Condition_Icy,0
Surface Condition_Snowy,0
Surface Condition_Unknown,0


In [64]:
q1df_new.head()

,Injury Severity,Weather_RAIN,Weather_SNOW,Weather_UNKNOWN,Light_DARK,Light_LIGHT,Light_Unknown,Surface Condition_Icy,Surface Condition_Snowy,Surface Condition_Unknown,Surface Condition_Wet
0,0.0,0,0,0,0,1,0,0,0,0,0
1,0.0,0,0,0,0,1,0,0,0,0,0
2,0.0,0,0,0,0,1,0,0,0,0,0
3,0.0,0,0,0,0,1,0,0,0,0,0
4,0.0,0,0,0,0,0,0,0,0,0,0


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

q1_X = q1df_new.drop(columns=["Injury Severity", "Light_DARK", "Light_LIGHT"])
q1_X = sm.add_constant(q1_X)
q1_y = q1df_new[["Injury Severity"]]


X_train, X_test, y_train, y_test = train_test_split(q1_X, q1_y, test_size=0.3, random_state=42)




# Fit the logistic regression model
model = sm.Logit(q1_y, q1_X)
result = model.fit()


Optimization terminated successfully.
         Current function value: 0.469700
         Iterations 7


In [67]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:        Injury Severity   No. Observations:               183145
Model:                          Logit   Df Residuals:                   183136
Method:                           MLE   Df Model:                            8
Date:                Sat, 30 Nov 2024   Pseudo R-squ.:                0.001729
Time:                        23:31:58   Log-Likelihood:                -86023.
converged:                       True   LL-Null:                       -86172.
Covariance Type:            nonrobust   LLR p-value:                 1.126e-59
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.5361      0.007   -226.058      0.000      -1.549      -1.523
Weather_RAIN                 -0.1596      0.029     -5.599      0.000      -0.215     

At 5% significance level, the below column is not significant.

*   **Light_DARK**
*   **Light_LIGHT**
